# k means ||

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/04 15:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=prova iniziale>

# Import data from sklearn (load in the disk)

This is going to be removed in the Cloud Veneto version


In [4]:
sc.setLogLevel("ERROR") # To hide warnings

In [5]:
!pip install scikit-learn # to be run at the launch of "docker compose up" 

In [6]:
import numpy as np
import pandas as pd
import random
import sklearn.datasets #va installato
import time

# from pyspark.sql.functions import sum
from pyspark.sql.functions import col
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.sql.functions import rand
from pyspark.sql.functions import least
from pyspark.sql import Row
from pyspark.ml.feature import StandardScaler

In [7]:
# Hyperparameters
k = 4
G = 3 # Giosu factor
len_df = 494_021

In [8]:
# Import dataframe directly from sklearn
prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

In [9]:
n_cols = len(X.iloc[0])

In [10]:
n_cols = n_cols - 3 #because we will remove the binary variables

In [11]:
# Show the dataframe (to be deleted for a bigger dataframe) (TO BE DELETED)
X

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,b'tcp',b'http',b'SF',310,1881,0,0,0,0,...,86,255,1.0,0.0,0.01,0.05,0.0,0.01,0.0,0.0
494017,0,b'tcp',b'http',b'SF',282,2286,0,0,0,0,...,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0
494018,0,b'tcp',b'http',b'SF',203,1200,0,0,0,0,...,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0
494019,0,b'tcp',b'http',b'SF',291,1200,0,0,0,0,...,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0


# Create a Spark dataframe
And then acquire important informations about the dataframe: # columns, # rows, # partitions, the schema, ...

In [12]:
# Create the spark dataframe with a smaller chunk of data (just to work easily)
X_smaller = X.iloc[random.sample(range(0, len(X.index)), len_df)]
spark_X = spark.createDataFrame(X_smaller)
spark_X.persist()

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[duration: bigint, protocol_type: binary, service: binary, flag: binary, src_bytes: bigint, dst_bytes: bigint, land: bigint, wrong_fragment: bigint, urgent: bigint, hot: bigint, num_failed_logins: bigint, logged_in: bigint, num_compromised: bigint, root_shell: bigint, su_attempted: bigint, num_root: bigint, num_file_creations: bigint, num_shells: bigint, num_access_files: bigint, num_outbound_cmds: bigint, is_host_login: bigint, is_guest_login: bigint, count: bigint, srv_count: bigint, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: bigint, dst_host_srv_count: bigint, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double]

In [13]:
# Check the number of partitions the DataFrame is divided into the three workers yet
spark_X.rdd.getNumPartitions()

3

In [14]:
# Define a function that is able to select the single row in the dataframe
def getrows(df, rownums=None): # copied from the internet
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])

In [15]:
# Define the number of rows and columns in the dataframe
n_rows = spark_X.count()

In [16]:
# Delete the non-numerical values in an easy way
# clean_X = spark_X.select('*').drop('protocol_type', 'service', 'flag')
# clean_X.printSchema()

# Delete the non-numerical values in a reusable way
col_type = np.array(spark_X.dtypes)
types = col_type[:,1] 
colnames = col_type[:,0]
clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])

# Step 0: Useful functions for the initialization

In [17]:
# Define the function for the distance between the cluster centers and the pandas dataframe
def distance(xrow, centers, num_cols=n_cols):
    '''Distance between a dataframe.row
    and the broadcasted value list of centers
    in the form of dataframe.row.value:
    
    broadC = sc.broadcast(center_rows).value
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)[:num_cols]
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])[:num_cols]
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.min(the_ds)

# Formula to evaluate the oversamping factor with the pre-factor G
def evaluate_l(log_phi, k, G):
    return G * k/log_phi # G = over-oversampling factor

# Function to select a row based on its probability
def select_row(x):
    if x > np.random.uniform(low = 0, high = 1):
        return True
    else:
        return False


### This is NOT $\Phi$. This is $log(\Phi)$. Pay attention budeo.

# Step 1: Choose a random sample from the dataset
This is the required step to begin the algorithm (doesn't need to be parallelized, since it is a select task)

In [18]:
# Choose the first sample randomly: select the random row
random_n = [np.random.randint(0, n_rows)]
random_sample = getrows(clean_X, random_n).collect()

# Step 2: Initial cost
Then we evaluate the cost function (sum of the squares after the first selection)

In [19]:
colnames = list(clean_X.dtypes[i][0] for i in range(len(clean_X.dtypes)))

In [20]:
# Add the colunm to keep the minimum distance between each point and the closest center
clean_X = (clean_X.select('*')
           .withColumn('minimum_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames)))

In [21]:
# Initial cost
initial_cost = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])
initial_cost

40.71877677868614

In [22]:
# Add the newly introduced value to the row that I selected in the first place
temp = random_sample[0].asDict()
temp["minimum_cost"] = 0.
random_sample[0] = Row(**temp)

In [23]:
# Broadcasting the row over the workers
bCent = sc.broadcast(random_sample)

# To show the content of the broadcast: b.value

## Step 2.5: evaluate number of iterations

After having evaluated $\log \phi$, we find a number of iterations which is of the same order of magnitude.

In [24]:
n_iter = int(initial_cost)
n_iter

40

# Step 3: For loop
Implement the for loop in order to evaluate probabilities and choose new centers



In [25]:
phi_iter = initial_cost
l = evaluate_l(phi_iter, k, G)
l

0.2947043341999725

In [26]:
%%time

i = 0
last_centers = 1
start_time = time.time()

while i < n_iter:   # or len(bCent.value) < k: # added as a safety condition (useless for very long datasets)

        '''
        Nel ciclo for:
            - Evaluate for each row l * d()^2 / phi
            - Sample with that probability
            - Broadcast centers to nodes
            - Evaluate new cost

        Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
        '''

        # Evaluate the probability and select the new rows
        mod_phi = l/np.exp(phi_iter)
        new_rows = clean_X.select('*').withColumn('random_number', rand(seed=int(time.time())))\
                          .filter(col('random_number') < col('minimum_cost')*mod_phi).drop('random_number').collect()

        print("--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()
        
        # Update the broadcast in a non-usual way (newest at the beginning)
        bCent = sc.broadcast(new_rows + bCent.value)
        
        print("--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()

        # Update the minimum distance
        if len(new_rows) == 1:
            clean_X = clean_X.select('*').\
                      withColumn('minimum_cost', least('minimum_cost', sum((col(colname)-bCent.value[0][colname])**2 for colname in colnames) )).cache()

        elif len(new_rows) > 1:
            clean_X = clean_X.select('*').withColumn('dummy', least(*[sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) for center in range(len(new_rows))] )).\
                      withColumn('minimum_cost', least('minimum_cost', 'dummy')).\
                      drop('dummy').cache()

        last_centers = len(bCent.value)

        # Evaluate new cost
        phi_iter = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])

        print("--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()

        i += 1

        print("\n at iteration", i ,", #values: ",len(bCent.value), " \n")


--- 1.6864678859710693 seconds ---
--- 0.018805503845214844 seconds ---


--- 0.9662654399871826 seconds ---

 at iteration 1 , #values:  1  



--- 1.547433614730835 seconds ---
--- 0.0230255126953125 seconds ---
--- 0.7714169025421143 seconds ---

 at iteration 2 , #values:  1  



--- 1.3605256080627441 seconds ---
--- 0.012633562088012695 seconds ---


--- 0.8368041515350342 seconds ---

 at iteration 3 , #values:  1  



...

--- 0.5941178798675537 seconds ---
--- 0.003421306610107422 seconds ---


--- 2.1732540130615234 seconds ---

 at iteration 30 , #values:  9  

--- 0.5382635593414307 seconds ---
--- 0.0032608509063720703 seconds ---


--- 2.7237448692321777 seconds ---

 at iteration 31 , #values:  11  

--- 0.6379945278167725 seconds ---
--- 0.007597923278808594 seconds ---
--- 0.440155029296875 seconds ---

 at iteration 32 , #values:  11  

--- 0.6306619644165039 seconds ---
--- 0.013180255889892578 seconds ---
--- 0.4781942367553711 seconds ---

 at iteration 33 , #values:  11  

--- 0.7306451797485352 seconds ---
--- 0.016397714614868164 seconds ---
--- 0.4692056179046631 seconds ---

 at iteration 34 , #values:  11  

--- 0.580040454864502 seconds ---
--- 0.013510704040527344 seconds ---
--- 0.42537474632263184 seconds ---

 at iteration 35 , #values:  11  

--- 0.703477144241333 seconds ---
--- 0.010349035263061523 seconds ---
--- 0.4785492420196533 seconds ---

 at iteration 36 , #values:  11  

--- 0.6486239433288574 seconds ---
--- 0.011703729629516602 seconds ---


--- 2.461651563644409 seconds ---

 at iteration 37 , #values:  12  

--- 0.6273152828216553 seconds ---
--- 0.002964496612548828 seconds ---
--- 0.3380565643310547 seconds ---

 at iteration 38 , #values:  12  

--- 0.5524320602416992 seconds ---
--- 0.004450798034667969 seconds ---
--- 0.3756387233734131 seconds ---

 at iteration 39 , #values:  12  

--- 0.6348567008972168 seconds ---
--- 0.014194726943969727 seconds ---


--- 2.1743760108947754 seconds ---

 at iteration 40 , #values:  13  

CPU times: user 989 ms, sys: 301 ms, total: 1.29 s
Wall time: 1min 15s


for each row evaluate distance from new points --> if it's less than mindistance substitute

In [27]:
clean_X.printSchema()

root
 |-- duration: long (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)
 |-- land: long (nullable = true)
 |-- wrong_fragment: long (nullable = true)
 |-- urgent: long (nullable = true)
 |-- hot: long (nullable = true)
 |-- num_failed_logins: long (nullable = true)
 |-- logged_in: long (nullable = true)
 |-- num_compromised: long (nullable = true)
 |-- root_shell: long (nullable = true)
 |-- su_attempted: long (nullable = true)
 |-- num_root: long (nullable = true)
 |-- num_file_creations: long (nullable = true)
 |-- num_shells: long (nullable = true)
 |-- num_access_files: long (nullable = true)
 |-- num_outbound_cmds: long (nullable = true)
 |-- is_host_login: long (nullable = true)
 |-- is_guest_login: long (nullable = true)
 |-- count: long (nullable = true)
 |-- srv_count: long (nullable = true)
 |-- serror_rate: double (nullable = true)
 |-- srv_serror_rate: double (nullable = true)
 |-- rerror_rate: double (nullable = true)
 |-- s

# Step 4: Select a subset of the possible centroids using k-means ++

Using kmeans ++:

Algorithm 1 k-means++(k) initialization.
1: C ← sample a point uniformly at random from X
2: while |C| < k do
2
3:
Sample x ∈ X with probability dφX(x,C)
(C)
4:
C ← C ∪ {

#### Function to find closest center among list

In [28]:
# Define the function to find the closest center from any point in order to find the weights
def find_closest_center(xrow, broadC, num_cols=n_cols):
    '''Find the center in the broadcasted value 
    list whose distance from a dataframe.row 
    is the lowest:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)[:num_cols]
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])[:num_cols]
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.argmin(the_ds)

In [29]:
dfCent = spark.createDataFrame(bCent.value) 

In [30]:
over_sampled_centers = dfCent.count()

In [31]:
dfCent.rdd.getNumPartitions()

3

In [32]:
len(bCent.value)

13

## Reduce size of dfCent to have only `k` centroids 

#### Step 1: calc weights $w_x$
set wx to be the number of points in X closer to x than any other point in C

In [33]:
wx = clean_X.rdd.map(lambda row: (find_closest_center(row,bCent),1)).reduceByKey(lambda x,y: x+y).\
     takeOrdered(over_sampled_centers) #o lambda x,y: x+y

In [34]:
wx

[(0, 7947),
 (1, 22),
 (2, 3),
 (3, 8),
 (4, 2343),
 (5, 12),
 (6, 5),
 (7, 24),
 (8, 18),
 (9, 68),
 (10, 59),
 (11, 1),
 (12, 483511)]

In [35]:
type(wx)

list

In [36]:
wx = (np.array(wx)[:,1]).astype(float)
wx

array([7.94700e+03, 2.20000e+01, 3.00000e+00, 8.00000e+00, 2.34300e+03,
       1.20000e+01, 5.00000e+00, 2.40000e+01, 1.80000e+01, 6.80000e+01,
       5.90000e+01, 1.00000e+00, 4.83511e+05])

In [37]:
wx /= np.sum(wx)
wx

array([1.60863607e-02, 4.45325199e-05, 6.07261635e-06, 1.61936436e-05,
       4.74271337e-03, 2.42904654e-05, 1.01210272e-05, 4.85809308e-05,
       3.64356981e-05, 1.37645971e-04, 1.19428121e-04, 2.02420545e-06,
       9.78725601e-01])

In [38]:
# Maybe we need to get rid of bCent ??
# bCent.destroy()
# bCent.unpersist()

#### Step 2: Use weighted `k-means++`
    2.1 Draw 1 random center
    2.2 update cost function
    2.3 repeat 1-2 until happy

In [39]:
# 2.1 Draw 1 random center

first_index = np.random.choice(a=range(wx.shape[0]), size=1, p=wx) 
#first point sampled uniformly wrt distance

first_index

array([12])

In [40]:
first_center = getrows(dfCent, first_index).collect()
first_center

[Row(duration=0, src_bytes=0, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=251, srv_count=16, serror_rate=1.0, srv_serror_rate=1.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=0.06, diff_srv_rate=0.06, srv_diff_host_rate=0.0, dst_host_count=255, dst_host_srv_count=16, dst_host_same_srv_rate=0.06, dst_host_diff_srv_rate=0.08, dst_host_same_src_port_rate=0.0, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=1.0, dst_host_srv_serror_rate=1.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0, minimum_cost=0.0)]

In [41]:
# Adds the first center to the ultimate center
bCent_ultimate = sc.broadcast(first_center)
bCent_ultimate.value

[Row(duration=0, src_bytes=0, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=251, srv_count=16, serror_rate=1.0, srv_serror_rate=1.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=0.06, diff_srv_rate=0.06, srv_diff_host_rate=0.0, dst_host_count=255, dst_host_srv_count=16, dst_host_same_srv_rate=0.06, dst_host_diff_srv_rate=0.08, dst_host_same_src_port_rate=0.0, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=1.0, dst_host_srv_serror_rate=1.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0, minimum_cost=0.0)]

In [42]:
# Define the total distance from other centers (minimum cost) over the chosen center
dfCent = (dfCent.select('*')
         .withColumn('minimum_cost', sum((col(colname)-first_center[0][colname])**2 for colname in colnames)))

In [43]:
# Initialize the total cost value
phi0 = np.log(dfCent.agg({"minimum_cost": "sum"}).collect()[0][0])
phi0

40.71430679588668

In [44]:
%%time

# 2.2 Update cost function

ultimate_sample_n = len(bCent_ultimate.value)

while ultimate_sample_n < k: #n_iter or len(bCent.value) < k:
    '''
    Nel ciclo for:
        - Evaluate for each row wx * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''
    
    # Evaluate the probability and select the new rows
    rows_prob = np.array(dfCent.rdd\
                .map(lambda row: distance(row,bCent_ultimate.value)).collect())
    # Forse pensare ad un modo alternativo di fare questo conto qui.... crescita lineare di bCent_ultimate
    
    # Sample new weighted random center
    another_index = np.random.choice(a=range(wx.shape[0]), size=1, p = rows_prob*wx/(rows_prob@wx) ) 
    another_center = getrows(dfCent, another_index).collect()
    
    # Update the broadcast
    bCent_ultimate = sc.broadcast(another_center + bCent_ultimate.value)
    
    # Update minimum cost
    dfCent = dfCent.select('*').\
    withColumn('minimum_cost', least('minimum_cost', sum((col(colname)-another_center[0][colname])**2 for colname in colnames) )).cache()
    
    # Evaluate new cost
    phi0 = np.log(dfCent.agg({"minimum_cost": "sum"}).collect()[0][0])
    
    ultimate_sample_n = len(bCent_ultimate.value)
    print("\n i: ",ultimate_sample_n)



 i:  2

 i:  3

 i:  4
CPU times: user 147 ms, sys: 30.8 ms, total: 178 ms
Wall time: 2.04 s


In [ ]:
'''
- Prendere la lista dei centroidi e farla diventare un distributed dataset
- Fare una NUOVA lista di centroidi 
- for i in range(k):
    pesca un singolo dato con probabilità d(x)/phi --> come fare?
    aggiungilo alla lista

'''

# TO DO:
    - Capire come scegliere l
    - Fare la normalizzazione dei dati (se la facciamo)
    - Capire se abbiamo ottimizzato nel modo giusto
    - Implementare tutto k means (optional)
    - Creare un file gemello con un dataframe visualizzabile e vedere se ha senso
    - Iniziare a fare i benchmark su cloud veneto
    - Grafici etc...

# Stop worker and master
Stop the running Spark context (sc) and Spark session (spark)

In [ ]:
# sc.stop()
# spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.